In [1]:
import pandas as pd
from pricer.pricer import Pricer
from pricer.pricer_factory import PricerFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
## initializing pricer_class
pricer_class = PricerFactory.build(Pricer.WEEKLY_STOCK_SPECULATION)

In [3]:
backtest_start_date = datetime(2016,1,1)
backtest_end_date = datetime(2023,1,1)

In [4]:
## creating database objects
market = Market()
sec = SEC()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [6]:
market.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = pricer_class.training_set(ticker,prices)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:34<00:00, 14.16it/s]


In [7]:
data = pd.concat(training_sets)

In [8]:
training_data = data.dropna().copy().sort_values(["year","week"])

In [9]:
training_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,year,week,ticker,adjclose,y
13,25.077294,24.083908,24.008833,22.553571,22.576964,23.736821,24.387693,23.652944,23.349112,24.923416,25.352133,26.085898,27.231891,26.158747,2000,14,MMM,25.077294,25.480959
13,2.011732,1.930327,1.888632,1.738618,1.778548,1.858409,1.942185,1.901427,1.986582,2.131963,2.311003,2.399871,2.383116,2.356966,2000,14,AOS,2.011732,2.101078
13,9.418513,8.651546,8.481721,8.125039,7.718261,8.309892,8.097987,8.604456,8.673087,8.325422,7.615064,8.143700,8.624169,8.669194,2000,14,ABT,9.418513,9.555718
13,19.338000,21.370000,24.750000,25.257000,31.413000,34.757000,36.436250,35.838000,28.851000,28.088000,29.488000,25.828750,20.457000,18.020000,2000,14,ABMD,19.338000,17.576000
13,0.761955,0.951261,0.987468,0.983182,1.035496,1.020570,0.966113,1.033427,1.087515,1.229384,1.166577,1.194988,1.265442,1.185050,2000,14,ATVI,0.761955,0.523290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,130.815000,130.980778,138.102000,137.546000,139.250000,139.020000,137.526000,132.838000,132.212500,129.322000,128.116000,126.726000,126.726000,127.656000,2023,22,YUM,130.815000,133.525000
1221,268.640000,267.430000,273.776000,270.246000,268.038000,284.972000,294.454000,303.330000,305.192500,300.448000,290.172000,289.752000,300.028000,302.866000,2023,22,ZBRA,268.640000,269.605000
1141,127.735000,129.932000,135.202000,136.870000,139.200000,138.614000,136.434000,131.202000,128.430000,126.920144,125.862528,124.938286,122.878206,123.457105,2023,22,ZBH,127.735000,130.370000
1221,28.235000,28.686000,25.505641,23.508000,23.418000,27.166000,30.916000,30.274000,28.712500,30.110000,29.658000,30.808000,44.718000,49.798000,2023,22,ZION,28.235000,29.420000


In [10]:
pricer_class.db.connect()
model_start_date = backtest_start_date - timedelta(pricer_class.time_horizon_class.model_date_offset)
model_end_date = backtest_end_date - timedelta(pricer_class.time_horizon_class.model_date_offset)
for year in range(model_start_date.year,model_end_date.year):
    training_slice = training_data[(training_data["year"]<year) & (training_data["year"] >= year - 4)].reset_index(drop=True)
    prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
    stuff = modeler_strat.model(training_slice,prediction_set,pricer_class.factors,False)
    stuff = stuff.rename(columns={"prediction":f"price_prediction"})
    relevant_columns = list(set(list(stuff.columns)) - set(pricer_class.factors))
    stuff = pricer_class.sim_processor(stuff)
    pricer_class.db.store("sim",stuff[relevant_columns])
pricer_class.db.disconnect()